In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup

import re

import numpy as np

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.worldometers.info/coronavirus/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[0]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
# df_bs.drop('S. No.', axis=1, inplace=True)

# there are 36 states+UT in India
df_bs.head(10)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"2,361,252","+5,797","140,392",+756,"991,231","+3,295","1,229,629","19,404",,,,,,North America,\n,,
1,,\nSouth America\n,"1,267,858",+637,"54,648",+25,"668,425","+42,652","544,785","11,987",,,,,,South America,\n,,
2,,\nEurope\n,"2,139,738",,"180,941",,"1,120,777",,"838,020","6,907",,,,,,Europe,\n,,
3,,\nAsia\n,"1,469,184","+6,988","37,203",+122,"891,933","+2,509","540,048","15,081",,,,,,Asia,\n,,
4,,\nAfrica\n,"212,092",+81,"5,718",+1,"97,185",+22,"109,189",503,,,,,,Africa,\n,,
5,,\nOceania\n,"8,896",+9,124,,"8,335",+3,437,3,,,,,,Australia/Oceania,\n,,
6,,\n\n,721,,15,,651,,55,4,,,,,,,\n,,
7,,World,"7,459,741","+13,512","419,041",+904,"3,778,537","+48,481","3,262,163","53,889",957,53.8,,,,All,\n,,
8,1,USA,"2,066,401",,"115,130",,"808,494",,"1,142,777","16,838","6,245",348,"22,625,758","68,377","330,896,410",North America,160,"2,874",15
9,2,Brazil,"775,184",,"39,797",,"396,692","+16,392","338,695","8,318","3,648",187,"1,364,423","6,422","212,475,738",South America,274,"5,339",156


In [6]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [7]:
df_bs.tail()

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
212,205,Seychelles,11,,,,11,,0,,112,,,,"98,314",Africa,"8,938",,
213,206,MS Zaandam,9,,2,,,,7,,,,,,,,,,
214,207,Western Sahara,9,,1,,6,,2,,15,2,,,"596,388",Africa,"66,265","596,388",
215,208,British Virgin Islands,8,,1,,7,,0,,265,33,167,"5,526","30,220",North America,"3,778","30,220",181
216,209,Papua New Guinea,8,,,,8,,0,,0.9,,"5,281",591,"8,936,574",Australia/Oceania,"1,117,072",,"1,692"


In [8]:
df_bs = df_bs.loc[8:].reset_index(drop=True)
df_bs = df_bs.drop('#', axis=1)
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,USA,"2,066,401",,"115,130",,"808,494",,"1,142,777","16,838","6,245",348,"22,625,758","68,377","330,896,410",North America,160,"2,874",15
1,Brazil,"775,184",,"39,797",,"396,692","+16,392","338,695","8,318","3,648",187,"1,364,423","6,422","212,475,738",South America,274,"5,339",156
2,Russia,"493,657",,"6,358",,"252,783",,"234,516","2,300","3,383",44,"13,545,303","92,820","145,931,211",Europe,296,"22,952",11
3,UK,"290,143",,"41,128",,N/A,N/A,N/A,516,"4,275",606,"6,042,622","89,037","67,866,605",Europe,234,"1,650",11
4,Spain,"289,360",,"27,136",,N/A,N/A,N/A,617,"6,189",580,"4,465,338","95,507","46,753,838",Europe,162,"1,723",10


In [9]:
df_bs = df_bs.iloc[:, :-3]

In [10]:
df_bs.columns

Index(['Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/\n1M pop\n',
       'Population', 'Continent'],
      dtype='object')

In [11]:
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"330,896,410","2,066,401",,"115,130",,"808,494",,"1,142,777","16,838","6,245",348,"22,625,758","68,377"
1,Brazil,South America,"212,475,738","775,184",,"39,797",,"396,692","+16,392","338,695","8,318","3,648",187,"1,364,423","6,422"
2,Russia,Europe,"145,931,211","493,657",,"6,358",,"252,783",,"234,516","2,300","3,383",44,"13,545,303","92,820"
3,UK,Europe,"67,866,605","290,143",,"41,128",,N/A,N/A,N/A,516,"4,275",606,"6,042,622","89,037"
4,Spain,Europe,"46,753,838","289,360",,"27,136",,N/A,N/A,N/A,617,"6,189",580,"4,465,338","95,507"


In [12]:
df_bs.columns[2:]

Index(['Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop'],
      dtype='object')

In [13]:
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)
df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['Mayotte', 'CAR', 'French Guiana', 'Diamond Princess',
       'Channel Islands', 'Réunion', 'Isle of Man', 'Martinique',
       'Faeroe Islands', 'Cayman Islands', 'Gibraltar', 'Guadeloupe',
       'Brunei ', 'Aruba', 'Sint Maarten', 'French Polynesia', 'Macao',
       'Saint Martin', 'St. Vincent Grenadines', 'Curaçao',
       'New Caledonia', 'Falkland Islands', 'Turks and Caicos',
       'Montserrat', 'MS Zaandam', 'British Virgin Islands'], dtype=object)

In [14]:
for col in df_bs.columns[2:]:
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)
 

df_bs.replace('', np.nan)

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,330896410,2066401,NaN,115130,NaN,808494,NaN,1142777,16838,6245,348,22625758,68377,Americas
1,Brazil,South America,212475738,775184,NaN,39797,NaN,396692,16392,338695,8318,3648,187,1364423,6422,Americas
2,Russia,Europe,145931211,493657,NaN,6358,NaN,252783,NaN,234516,2300,3383,44,13545303,92820,Europe
3,UK,Europe,67866605,290143,NaN,41128,NaN,NaN,NaN,NaN,516,4275,606,6042622,89037,Europe
4,Spain,Europe,46753838,289360,NaN,27136,NaN,NaN,NaN,NaN,617,6189,580,4465338,95507,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Seychelles,Africa,98314,11,NaN,NaN,NaN,11,NaN,0,NaN,112,NaN,NaN,NaN,Africa
205,MS Zaandam,NaN,NaN,9,NaN,2,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN
206,Western Sahara,Africa,596388,9,NaN,1,NaN,6,NaN,2,NaN,15,2,NaN,NaN,Africa
207,British Virgin Islands,North America,30220,8,NaN,1,NaN,7,NaN,0,NaN,265,33,167,5526,NaN


In [15]:
df_bs.to_csv('worldometer_data.csv', index=False)